In [2]:
import pandas as pd
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


Извлечем и визуализируем наши временные ряды

In [ ]:
def retrieve_time_series(api, series_ID):
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    df = pd.DataFrame(series_search)
    return df

In [ ]:
def plot_data(df, x_variable, y_variable, title):
    fig, ax = plt.subplots()
    ax.plot_date(df[x_variable], 
                 df[y_variable], marker='', linestyle='-', label=y_variable)
    fig.autofmt_xdate()
    plt.title(title)
    plt.show()

In [ ]:
#Create EIA API using your specific API key
api_key = "YOUR EIA API KEY HERE"
api = eia.API(api_key)
#Declare desired series ID
series_ID='TOTAL.GEEGPUS.M'
df = retrieve_time_series(api, series_ID)
df.reset_index(level=0, inplace=True)
df.rename(columns={'index':'Date',
                   df.columns[1]:'Geothermal_net_generation'}, inplace=True)
#Convert the Date column into a date object
df['Date'] = df['Date'].str.rstrip()
df['Date'] = df['Date'].str.replace(' ', '-')
df['Date']=pd.to_datetime(df['Date'], format='%Y-%m')
#Plot the time series
plot_data(df, 'Date', 
          'Geothermal_net_generation', 
          'Net Generation for Geothermal over Time')

Прежде чем мы создадим модель SARIMA, разложим временные ряды, чтобы убедиться, что они отображают сезонность. Мы используем функцию seasonal_decompose, доступную через пакет statsmodels.tsa. Эта функция позволяет нам разбивать временные ряды на трендовые, сезонные и остаточные компоненты:

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

def decompose_time_series(series, frequency):
    """
    Decompose a time series and plot it in the console
    Arguments: 
        series: series. Time series that we want to decompose
    Outputs: 
        Decomposition plot in the console
    """
    result = seasonal_decompose(series, model='additive', freq = frequency)
    result.plot()
    plt.show()

### EXECUTE IN MAIN FUNCTION ###
#Decompose the time series to determine seasonality/trend
decompose_time_series(df['Geothermal_net_generation'], 12)

Давайте сначала разделим наши данные на обучающие и тестовые наборы. Таким образом, мы можем построить нашу модель с использованием обучающего набора и оценить ее производительность с использованием тестовых данных:

In [ ]:
def time_series_train_test_split(time_series, train_split_fraction):
    """
    Split the data into training and test set.
    """
    split_index = int(round(time_series.shape[0]*train_split_fraction, 0))
    train_set = time_series[:split_index]
    test_set = time_series[:-split_index]
    return train_set, test_set

### EXECUTE IN MAIN FUNCTION ###
training_set, test_set = time_series_train_test_split(time_series = df['Geothermal_net_generation'], 
                                                      train_split_fraction = .75)

Построим нашу модель SARIMA. Мы перебираем каждую из возможных конфигураций гиперпараметров, генерируя модель SARIMA. Мы используем параметры AIC для каждой возможной модели для оценки производительности модели. Нам нужна модель с наименьшими оценками AIC:

In [ ]:
def seasonal_arima_model(time_series, order, seasonal_order, trend):
    """
    Generate a seasonal ARIMA model using a set of hyperparameters. Returns the model fit, and the 
    associated model AIC and BIC values.
    """ 
    try:
        model = sm_api.tsa.SARIMAX(time_series, 
                                   order=order, 
                                   seasonal_order=seasonal_order, 
                                   trend = trend,
                                   enforce_stationarity=False, 
                                   enforce_invertibility=False)
        model_fit = model.fit()
        #Print the model results
        print(model_fit.summary())
        return model_fit, model_fit.aic, model_fit.bic
    except:
        print("Could not fit with the designated model parameters")
        return None, None, None

### EXECUTE IN MAIN FUNCTION ###
lowest_aic_val = 100000000000
#Generate  model for each of hyperparameter combination in a loop
for order_combo in order_combos:
    for seasonal_order_combo in seasonal_order_combos:
        #Convert the combination to list format
        seasonal_order_combo = list(seasonal_order_combo)
        #Generate the SARIMA model
        model_fit, model_aic, model_bic = seasonal_arima_model(time_series = training_set, 
                                                    order = order_combo, 
                                                    seasonal_order = seasonal_order_combo[0:4],
                                                    trend = seasonal_order_combo[-1])
        #Test model performance, and keep running tab of best performing model
        #Set with the newest value if the lowest_aic_value hasn't yet been calculated (on first run),
        #or if the newly calculated model AIC is lower than the lowest calculated AIC value
        if (model_aic < lowest_aic_val):
            lowest_aic_val = model_aic
            best_model = model_fit
            best_order = order_combo
            best_seasonal_order = seasonal_order_combo
#Print the best model parameters after the 
print("Best model paramaters: order-- ", best_order, ", seasonal order-- ", best_seasonal_order)

Глядя на термин ‘P> abs (z)’ на графике, все переменные считываются как 0. Это здорово, так как мы хотим, чтобы наши значения P были как можно ближе к 0. Используя отсечение <0,05 для статистической значимости, все наши запаздывающие термины AR и MA значительно влияют на прогноз модели.

В приведенном ниже фрагменте кода мы прогнозируем значения для набора тестов, прогнозируя общее количество шагов, которые присутствуют в нашем наборе тестов. Затем мы сравниваем прогнозируемые значения с фактическими значениями, используя показатели среднеквадратичной ошибки (RMSE) и средней абсолютной ошибки (MAE). Чем ниже оценки RMSE и MAE, тем лучше подходит модель. Если прогнозируемые значения точно соответствуют фактическим значениям, оба RMSE и MAE равны 0.

In [ ]:
def fit_predictions(model_fit, steps_out_to_predict, actual_values):
    """
    This function predicts the SARIMA model out a certain designated number of steps,
    and compares the predictions to the actual values. The root mean squared error and
    the mean absolute error are calculated, comparing the predicted and actual values.
    The function returns the predicted values and their respective confidence intervals.
    Args:
        model_fit:  SARIMA model.
        steps_out_to_predict: Int. Number of steps out to predict the time series.
        actual_values: Series of actual time series values.
    Outputs:
        mean_predicted_values: Series of predicted time series values.
        confidence_interval_predicted_values: Dataframe, containing upper and lower thresholds of the
        confidence interval
    """
    predicted_values = model_fit.get_forecast(steps=steps_out_to_predict)
    mean_predicted_values = predicted_values.predicted_mean
    confidence_interval_predicted_values = predicted_values.conf_int()
    #Compare the actual to the predicted values using RMSE and MAE metrics
    rmse, mae = quantify_rmse_mae(mean_predicted_values, actual_values)
    print("Root mean squared error: ", str(rmse))
    print("Mean absolute error: ", str(mae))
    return mean_predicted_values, confidence_interval_predicted_values
    
def quantify_rmse_mae(predicted_values, actual_values):
    """
    This function calculates the root mean squared error and mean absolute error for 
    the predicted values, when compared to the actual values. These helps help us to
    gauge model performance. 
    Args:
        predicted_values: Series of predicted time series values.
        actual_values: Corresponding series of actual time series values.
    Outputs:
        rmse: Float. Root mean squared error.
        mae: Float. Mean absolute error.
    """
    #calcuate the mean squared error of the model
    rmse = math.sqrt(mean_squared_error(actual_values, predicted_values))
    #Calculate the mean absolute error of the model 
    mae = mean_absolute_error(actual_values, predicted_values)
    #Return the MSE and MAE for the model
    return rmse, mae
  
### EXECUTE IN THE MAIN FUNCTION ###
#Run the data on the test set to gauge model performance
mean_predicted_values, confidence_interval_predicted_values = fit_predictions(best_model, 
                                                                              len(test_set), 
                                                                              test_set)

При выполнении приведенного выше фрагмента кода выводимые значения RMSE и MAE для модели составляют приблизительно 868 и 860 соответственно.
После вычисления показателей RMSE и MAE мы сопоставляем прогнозируемые значения с фактическими значениями. Это дает нам наглядную оценку производительности модели:

In [ ]:
def plot_results(mean_predicted_values, confidence_interval_predicted_values, time_series):
    """
    This function plots actual time series data against SARIMA model-predicted values. 
    We include the confidence interval for the predictions. 
    Args:
        mean_predicted_values: Series of float values. The model-predicted values.
        confidence_interval_predicted_values: Pandas dataframe, containing the lower and
        upper confidence intervals.
        time_series: Series of float values. Actual time series values that we want to graph
    Outputs:
        None. Plot of the time series values, as well as the predicted values and associated 
        confidence interval.
    """
    ax = time_series.plot(label='Observed')
    mean_predicted_values.plot(ax=ax, label = 'Forecast', alpha=.7, figsize=(14, 4))
    ax.fill_between(confidence_interval_predicted_values.index,
                    confidence_interval_predicted_values.iloc[:, 0],
                    confidence_interval_predicted_values.iloc[:, 1], color='k', alpha=.2)
    ax.set_xlabel('Date Index')
    ax.set_ylabel('Value')
    plt.legend()
    plt.show()

### EXECUTE IN MAIN FUNCTION ###
#Plot the predictions against the real data
plot_results(mean_predicted_values, 
             confidence_interval_predicted_values, 
             df['Geothermal_net_generation'][400:])

Судя по приведенному выше графику, модель отлично справляется с прогнозированием временных рядов на 140 временных шагов.